In [1]:
import pandas as pd
from pandas import DataFrame
import os
import os.path as op
import sys
import csv
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
plt.show()
import glob
import seaborn as sns
import shutil
import networkx as nx

pd.set_option("display.max_rows", 6)

sys.path.insert(0, '/home/jmunson-mcgee/')
from JMM_functions import *

outdir='/mnt/scgc/simon/microg2p/analyses/GORG_recruitment/CoverM_comparsion'
safe_make_dir(outdir)
rundir='/mnt/scgc/simon/microg2p/analyses/GORG_recruitment/CoverM_comparsion'
os.chdir(rundir)

images='/mnt/scgc/simon/microg2p/analyses/GORG_recruitment/CoverM_comparsion'
safe_make_dir(images)
    
matplotlib.__version__

Folder already exists
Folder already exists


'3.1.0'

In [7]:
coverm=glob.glob('/mnt/scgc/simon/microg2p/analyses/GORG_recruitment/CoverM_comparsion/*GoM_SAGs.tsv')

SAGdata='/mnt/scgc/simon/microg2p/analyses/20210325_GoM_recluster/Summary_files/FINAL_GoM_Metadata.csv'


In [10]:
cols=['name', 'checkM_estimated_completeness', 'final_assembly_length', 'GTDB_classification']
SAG_data=pd.read_csv(SAGdata, usecols=cols)

SAG_data=SAG_data[SAG_data['checkM_estimated_completeness']!= 0.0]
SAG_data=SAG_data[SAG_data['GTDB_classification']!= 'Unclassified']
SAG_class=SAG_data[['name', 'GTDB_classification']].copy()

genome_size=SAG_data.groupby('GTDB_classification', as_index=False)['final_assembly_length', 'checkM_estimated_completeness'].mean()

count=SAG_data.groupby('GTDB_classification', as_index=False)['name'].count()
count=count.rename(columns={'name':'count'})
genome_size=genome_size.merge(count, on='GTDB_classification', how='left')


genome_size['estimated_total_genome_size']=genome_size['final_assembly_length']*(100/genome_size['checkM_estimated_completeness'])

# limit the genera that I make this calculation to only be the ones with at least 3 cells
genome_size=genome_size[genome_size['count']>=3]
genome_size['relative_genome_size']=genome_size['estimated_total_genome_size']/genome_size['estimated_total_genome_size'].mean()
genome_size.to_csv(os.path.join(outdir, 'genome_size_calculations.csv'))

avg_completeness=genome_size['checkM_estimated_completeness'].mean()
avg_genome_size=(genome_size['estimated_total_genome_size'].mean())

rel_genome_size=genome_size[['GTDB_classification', 'relative_genome_size', 'checkM_estimated_completeness', 
                             'estimated_total_genome_size']].copy()
rel_genome_size

,GTDB_classification,relative_genome_size,checkM_estimated_completeness,estimated_total_genome_size
1,AAA164-E04,2.025597,23.349143,5.458512e+06
2,AAA536-G10,0.936774,39.525536,2.524389e+06
3,AG-337-I02,0.741080,46.064667,1.997037e+06
...,...,...,...,...
519,d__Bacteria;p__Proteobacteria;c__Gammaproteoba...,0.870133,38.313333,2.344806e+06
521,d__Bacteria;p__SAR324;c__SAR324;o__SAR324;f__N...,1.280005,28.592500,3.449315e+06
526,d__Bacteria;p__Verrucomicrobiota;c__Verrucomic...,1.585405,45.960000,4.272298e+06


In [12]:
coverm_df=pd.DataFrame()


counter=1
for file in coverm:
    if counter==1:
        coverm_df=pd.read_csv(file, sep="\t")
        counter=2
    else:
        df=pd.read_csv(file, sep="\t")
        coverm_df=coverm_df.merge(df, on='Genome', how='outer')

# slice genomes to only incude the SAG identifier    
coverm_df['name']=coverm_df['Genome'].str[:10]    
coverm_df
# merge with the GTDB genus     
coverm_df=coverm_df.merge(SAG_class, on='name', how='left')
  


# move the column to head of list using index, pop and insert
cols = list(coverm_df)
cols.insert(0, cols.pop(cols.index('GTDB_classification')))
cols.insert(0, cols.pop(cols.index('name')))
coverm_df = coverm_df.loc[:, cols]
    
coverm_df=coverm_df.drop(columns='Genome')
coverm_df['GTDB_classification']=coverm_df['GTDB_classification'].fillna('Other')
coverm_df



,name,GTDB_classification,ALL_20190709_contf_pe_bbmerge_reads.fastq.gz Relative Abundance (%),ALL_20190402_contf_pe_bbmerge_reads.fastq.gz Relative Abundance (%),ALL_20181030_contf_pe_bbmerge_reads.fastq.gz Relative Abundance (%),ALL_20171130_contf_pe_bbmerge_reads.fastq.gz Relative Abundance (%)
0,unmapped,Other,46.937733,43.552130,52.473760,51.032830
1,AH-269-F19,SCGC-AAA160-P02,0.008955,0.000693,0.008857,0.066070
2,AH-135-A01,Pelagibacter,0.023162,0.006670,0.005012,0.004562
...,...,...,...,...,...,...
7516,AM-384-P22,UBA10364,0.010743,0.004844,0.018268,0.042606
7517,AM-384-P23,UBA8337,0.000006,0.000000,0.000391,0.000430
7518,AM-382-D14,JAACDM01,0.001061,0.000019,0.005519,0.002545


In [13]:
#generate a list of just the columns representing metagenomes
cols = list(coverm_df)
cols=cols[2:]
cols

['ALL_20190709_contf_pe_bbmerge_reads.fastq.gz Relative Abundance (%)',
 'ALL_20190402_contf_pe_bbmerge_reads.fastq.gz Relative Abundance (%)',
 'ALL_20181030_contf_pe_bbmerge_reads.fastq.gz Relative Abundance (%)',
 'ALL_20171130_contf_pe_bbmerge_reads.fastq.gz Relative Abundance (%)']

In [17]:
abundance=coverm_df.groupby(['GTDB_classification'], as_index=False)[cols].sum()
abundance.to_csv(os.path.join(outdir, 'coverm_all_genera_relative_abundance_90perc_ID.csv'))

abundance=abundance.merge(rel_genome_size, on='GTDB_classification', how='left')

# replace the genus with "Other" for all lineages where I did not calculate an estimated genomes size because there are <3 cells
abundance['GTDB_classification']=np.where(abundance['relative_genome_size'].isnull(), 'Other', abundance['GTDB_classification'])

# redo the reative abundance
abundance=abundance.groupby(['GTDB_classification'], as_index=False)[cols].sum()

abundance=abundance.merge(rel_genome_size, on='GTDB_classification', how='left')
abundance['relative_genome_size']=abundance['relative_genome_size'].fillna(1)

SAR11_rel=abundance[abundance['GTDB_classification'].str.contains('Pelagibacter')]

abundance.to_csv(os.path.join(outdir, 'coverm_genus_relative_abundance_90perc_ID.csv'))
abundance

,GTDB_classification,ALL_20190709_contf_pe_bbmerge_reads.fastq.gz Relative Abundance (%),ALL_20190402_contf_pe_bbmerge_reads.fastq.gz Relative Abundance (%),ALL_20181030_contf_pe_bbmerge_reads.fastq.gz Relative Abundance (%),ALL_20171130_contf_pe_bbmerge_reads.fastq.gz Relative Abundance (%),relative_genome_size,checkM_estimated_completeness,estimated_total_genome_size
0,AAA164-E04,0.001630,0.001096,0.045917,0.013966,2.025597,23.349143,5.458512e+06
1,AAA536-G10,0.176623,0.243574,0.434991,0.397502,0.936774,39.525536,2.524389e+06
2,AG-337-I02,0.000771,0.002253,0.158285,0.022984,0.741080,46.064667,1.997037e+06
...,...,...,...,...,...,...,...,...
208,d__Bacteria;p__Proteobacteria;c__Gammaproteoba...,0.000000,0.000011,0.008736,0.000106,0.870133,38.313333,2.344806e+06
209,d__Bacteria;p__SAR324;c__SAR324;o__SAR324;f__N...,0.000125,0.002979,0.006860,0.000376,1.280005,28.592500,3.449315e+06
210,d__Bacteria;p__Verrucomicrobiota;c__Verrucomic...,0.000034,0.000012,0.002043,0.000439,1.585405,45.960000,4.272298e+06


In [18]:

# Now I need to divide by the relative genome size
abundance[cols]=abundance[cols].div(abundance.relative_genome_size, axis=0)

# and finally divide each value by the sum of the column it is in and multiple by 100
# sums=abundance[cols].sum()
abundance[cols]=abundance[cols]/abundance[cols].sum()*100
abundance.to_csv(os.path.join(outdir, 'coverm_genus_relative_abundance_genome_size_normalized_90perc_ID.csv'))

SAR11_norm=abundance[abundance['GTDB_classification'].str.contains('Pelagibacter')]

abundance

,GTDB_classification,ALL_20190709_contf_pe_bbmerge_reads.fastq.gz Relative Abundance (%),ALL_20190402_contf_pe_bbmerge_reads.fastq.gz Relative Abundance (%),ALL_20181030_contf_pe_bbmerge_reads.fastq.gz Relative Abundance (%),ALL_20171130_contf_pe_bbmerge_reads.fastq.gz Relative Abundance (%),relative_genome_size,checkM_estimated_completeness,estimated_total_genome_size
0,AAA164-E04,0.000734,0.000502,0.019880,0.006507,2.025597,23.349143,5.458512e+06
1,AAA536-G10,0.171999,0.241164,0.407226,0.400478,0.936774,39.525536,2.524389e+06
2,AG-337-I02,0.000950,0.002820,0.187312,0.029271,0.741080,46.064667,1.997037e+06
...,...,...,...,...,...,...,...,...
208,d__Bacteria;p__Proteobacteria;c__Gammaproteoba...,0.000000,0.000011,0.008805,0.000115,0.870133,38.313333,2.344806e+06
209,d__Bacteria;p__SAR324;c__SAR324;o__SAR324;f__N...,0.000089,0.002159,0.004700,0.000277,1.280005,28.592500,3.449315e+06
210,d__Bacteria;p__Verrucomicrobiota;c__Verrucomic...,0.000020,0.000007,0.001130,0.000261,1.585405,45.960000,4.272298e+06


In [19]:
SAR11_norm

,GTDB_classification,ALL_20190709_contf_pe_bbmerge_reads.fastq.gz Relative Abundance (%),ALL_20190402_contf_pe_bbmerge_reads.fastq.gz Relative Abundance (%),ALL_20181030_contf_pe_bbmerge_reads.fastq.gz Relative Abundance (%),ALL_20171130_contf_pe_bbmerge_reads.fastq.gz Relative Abundance (%),relative_genome_size,checkM_estimated_completeness,estimated_total_genome_size
69,Pelagibacter,10.144697,7.460345,9.505372,4.012867,0.508892,50.612276,1.371346e+06
70,Pelagibacter_A,0.120178,0.394918,1.005482,0.589150,0.447745,49.246981,1.206570e+06
71,Pelagibacteraceae,0.000413,0.002901,0.006262,0.062052,0.598134,32.516667,1.611832e+06
...,...,...,...,...,...,...,...,...
188,d__Bacteria;p__Proteobacteria;c__Alphaproteoba...,0.000014,0.000007,0.000030,0.000006,0.896587,20.753333,2.416094e+06
189,d__Bacteria;p__Proteobacteria;c__Alphaproteoba...,0.000037,0.000068,0.000219,0.000029,0.530596,38.756667,1.429832e+06
190,d__Bacteria;p__Proteobacteria;c__Alphaproteoba...,0.002839,0.011117,0.040482,0.018078,0.392090,57.198889,1.056592e+06


In [20]:
SAR11_norm.sum()

GTDB_classification                                                    PelagibacterPelagibacter_APelagibacteraceaed__...
ALL_20190709_contf_pe_bbmerge_reads.fastq.gz Relative Abundance (%)                                              10.4656
ALL_20190402_contf_pe_bbmerge_reads.fastq.gz Relative Abundance (%)                                              8.24109
                                                                                             ...                        
relative_genome_size                                                                                             6.52492
checkM_estimated_completeness                                                                                    533.173
estimated_total_genome_size                                                                                  1.75831e+07
Length: 8, dtype: object